In [46]:
import pandas as pd

data = pd.read_csv('competition_data.csv')
submission = pd.read_csv('submission.csv')

In [47]:
#eliminamos datos particulares de Chona, son siempre los mismos, y los que consideramos irrelevanets
data = data.drop(columns=['username', 'conn_country', 'user_agent_decrypted'])
submission = submission.drop(columns=['username', 'conn_country', 'user_agent_decrypted'])

In [48]:
# del timestamp, solo nos interesa la hora, el dia de la semana y el mes

data['ts'] = pd.to_datetime(data['ts'])
submission['ts'] = pd.to_datetime(submission['ts'])

# Extraer la hora
data['hour'] = data['ts'].dt.hour
submission['hour'] = submission['ts'].dt.hour

# Extraer el día de la semana (0 es lunes, 6 es domingo)
data['day_of_week'] = data['ts'].dt.dayofweek
submission['day_of_week'] = submission['ts'].dt.dayofweek

# Extraer el mes
data['month'] = data['ts'].dt.month
submission['month'] = submission['ts'].dt.month

# dropeamos ts
data = data.drop(columns=['ts'])
submission = submission.drop(columns=['ts'])

In [49]:
# del dispositivo, solo queremos saber si se reprodujo en celular (iphone) o no
data['is_iphone'] = data['platform'].apply(lambda x: 1 if ('ios' in x or 'iOS' in x) else 0)
submission['is_iphone'] = submission['platform'].apply(lambda x: 1 if ('ios' in x or 'iOS' in x) else 0)

# dropeamos platform
data = data.drop(columns=['platform'])
submission = submission.drop(columns=['platform'])

In [37]:
data

,Unnamed: 0,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,shuffle,TARGET,hour,day_of_week,month,is_iphone
0,110163,Crazy For U,Big Time Rush,24/seven,spotify:track:3jFfr89lnSmb4QBtfG8JBP,clickrow,False,False,5,6,3,1
1,66026,Nada Personal - Remasterizado 2007,Soda Stereo,Me Verás Volver (Hits & Más),spotify:track:09TTeexnlKewZdjOak2sV2,trackdone,True,False,10,0,6,1
2,116790,Good Times,CHIC,Risqué,spotify:track:0G3fbPbE1vGeABDEZF0jeG,trackdone,True,True,2,6,7,1
3,18431,Verano del 92,Los Piojos,3er Arco,spotify:track:1NXvuBAq48QrxRFQZVmORQ,trackdone,False,True,4,6,9,1
4,82941,Simpatico,Ekko Park,Simpatico,spotify:track:2gYJY0sIx1ErgTIha2nPRg,trackdone,True,False,18,3,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...
100139,78756,Losing My Religion,R.E.M.,In Time: The Best Of R.E.M. 1988-2003,spotify:track:12axV6NUqaYH3yFUWwArzr,trackdone,True,False,20,3,4,1
100140,12585,Cerca De La Revolucion,Charly García,Piano Bar,spotify:track:66grIvFLGrI4tNhggO2DAd,trackdone,True,False,21,2,11,1
100141,93960,A Los Jóvenes De Ayer - Remastered 2012,Serú Girán,Bicicleta,spotify:track:6YAl320SfxLO4rVIZTFKqG,trackdone,True,False,1,4,1,1
100142,74339,heat not hot,Serengeti,heat not hot,spotify:track:0rKLO1hXpmoIthQgJKoczN,trackdone,True,False,17,2,5,1


In [50]:
# Aplicar One-Hot Encoding a razon de reproduccion
reason_start_encoded = pd.get_dummies(data['reason_start'], prefix='reason', dummy_na=True)

# Concatenar el DataFrame original con el nuevo DataFrame de variables codificadas
data = pd.concat([data, reason_start_encoded], axis=1)

# Dropear la columna original
data = data.drop(columns=['reason_start'])

In [51]:
# Lista de columnas de One-Hot Encoding creadas previamente
encoded_columns = reason_start_encoded.columns


# convertir a entero los bools
for col in encoded_columns:
    data[col] = data[col].astype(int)

In [52]:
# Aplicar One-Hot Encoding a razon de reproduccion
reason_start_encoded = pd.get_dummies(submission['reason_start'], prefix='reason', dummy_na=True)

# Concatenar el DataFrame original con el nuevo DataFrame de variables codificadas
submission = pd.concat([submission, reason_start_encoded], axis=1)

# Dropear la columna original
submission = submission.drop(columns=['reason_start'])

In [53]:
# Lista de columnas de One-Hot Encoding creadas previamente
encoded_columns = reason_start_encoded.columns


# convertir a entero los bools
for col in encoded_columns:
    submission[col] = submission[col].astype(int)

In [54]:
data

,Unnamed: 0,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,shuffle,TARGET,hour,day_of_week,month,...,reason_appload,reason_backbtn,reason_clickrow,reason_fwdbtn,reason_playbtn,reason_remote,reason_trackdone,reason_trackerror,reason_unknown,reason_nan
0,110163,Crazy For U,Big Time Rush,24/seven,spotify:track:3jFfr89lnSmb4QBtfG8JBP,False,False,5,6,3,...,0,0,1,0,0,0,0,0,0,0
1,66026,Nada Personal - Remasterizado 2007,Soda Stereo,Me Verás Volver (Hits & Más),spotify:track:09TTeexnlKewZdjOak2sV2,True,False,10,0,6,...,0,0,0,0,0,0,1,0,0,0
2,116790,Good Times,CHIC,Risqué,spotify:track:0G3fbPbE1vGeABDEZF0jeG,True,True,2,6,7,...,0,0,0,0,0,0,1,0,0,0
3,18431,Verano del 92,Los Piojos,3er Arco,spotify:track:1NXvuBAq48QrxRFQZVmORQ,False,True,4,6,9,...,0,0,0,0,0,0,1,0,0,0
4,82941,Simpatico,Ekko Park,Simpatico,spotify:track:2gYJY0sIx1ErgTIha2nPRg,True,False,18,3,7,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100139,78756,Losing My Religion,R.E.M.,In Time: The Best Of R.E.M. 1988-2003,spotify:track:12axV6NUqaYH3yFUWwArzr,True,False,20,3,4,...,0,0,0,0,0,0,1,0,0,0
100140,12585,Cerca De La Revolucion,Charly García,Piano Bar,spotify:track:66grIvFLGrI4tNhggO2DAd,True,False,21,2,11,...,0,0,0,0,0,0,1,0,0,0
100141,93960,A Los Jóvenes De Ayer - Remastered 2012,Serú Girán,Bicicleta,spotify:track:6YAl320SfxLO4rVIZTFKqG,True,False,1,4,1,...,0,0,0,0,0,0,1,0,0,0
100142,74339,heat not hot,Serengeti,heat not hot,spotify:track:0rKLO1hXpmoIthQgJKoczN,True,False,17,2,5,...,0,0,0,0,0,0,1,0,0,0


In [55]:
#pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import webbrowser

In [56]:
# Configura el objeto SpotifyOAuth
auth_manager = SpotifyOAuth(
    client_id="bd28f888d5594351a74597b8c4750b07",
    client_secret="2ad3d9ea7c00425792697d028c5afbd5",
    redirect_uri="http://127.0.0.1:1234/",
    scope="user-top-read",
    open_browser=False
)

# Obtener el URL de autenticación y abrirlo en un navegador
auth_url = auth_manager.get_authorize_url()
print("Por favor, visita este URL para autorizar:", auth_url)
webbrowser.open(auth_url)

# Espera a que el usuario ingrese el código de redirección URL
response = input("Pega el URL completo al que fuiste redirigido aquí: ")
code = auth_manager.parse_response_code(response)
token_info = auth_manager.get_access_token(code)

if token_info:
    print("Token de acceso obtenido:", token_info['access_token'])
else:
    print("No se pudo obtener el token de acceso.")

Por favor, visita este URL para autorizar: https://accounts.spotify.com/authorize?client_id=bd28f888d5594351a74597b8c4750b07&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A1234%2F&scope=user-top-read


C:\Users\dafyd\AppData\Local\Temp\ipykernel_31664\2871716197.py:18: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = auth_manager.get_access_token(code)


Token de acceso obtenido: BQCAJXJXkxAJ8SMIPdTu0vpRi1pEDqepWIg70klruaaIpYdf8GHIiBcaNthqn-PP_hgEGA5udn1-j5kY_X2PnBwhfaG9ttW_ffnFzIJ-SdrmzKTynnF3LneJjF69_z0R_4JPrbgIN1OsvEG6zFmuVJkjHVK5MeeritqjJlKqk_XORF2igNTOUr5WwWiWGEE-VfhPNZLGn1iNWKMZ1tSVvnCfZuxJpCgSKj8xNdD8FwimRE2kcin52LUlx4tGhbrmvdNKsTfMyQY


In [57]:
sp = spotipy.Spotify(auth_manager=auth_manager)

In [58]:
def chunk_list(lst, size):
    """Divide la lista 'lst' en sublistas de tamaño 'size'."""
    return [lst[i:i + size] for i in range(0, len(lst), size)]

# Obtener los URIs únicos y válidos
tracks = data['spotify_track_uri'].dropna().unique()  # Elimina NaN directamente
tracks = [uri for uri in tracks if isinstance(uri, str)]  # Filtra que sean strings

# Dividir en chunks de hasta 50 elementos
chunks = chunk_list(tracks, 50)

In [59]:
uri_to_duration_popularity = {}
for i, chunk in enumerate(chunks):
    print(i)
    try:
        tracks_info = sp.tracks(chunk)  # Consulta a la API
        for track in tracks_info.get('tracks', []):
            if track:
                uri = track.get('uri')
                duration = track.get('duration_ms')
                popularity = track.get('popularity')
                uri_to_duration_popularity[uri] = (duration, popularity)
            else:
                print("⚠️  Track no encontrado o no disponible.")
    except Exception as e:
        print(f"❌ Error al procesar el chunk {i + 1}: {e}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204


In [60]:
# Definimos funciones auxiliares que buscan en el diccionario por cada fila
def get_duration(uri):
    return uri_to_duration_popularity.get(uri, (None, None))[0]

def get_popularity(uri):
    return uri_to_duration_popularity.get(uri, (None, None))[1]

# Creamos las nuevas columnas aplicando las funciones
data['duration_ms'] = data['spotify_track_uri'].apply(get_duration)
data['popularity'] = data['spotify_track_uri'].apply(get_popularity)

In [61]:
# Obtener los URIs únicos y válidos
tracks = submission['spotify_track_uri'].dropna().unique()  # Elimina NaN directamente
tracks = [uri for uri in tracks if isinstance(uri, str)]  # Filtra que sean strings

# Dividir en chunks de hasta 50 elementos
chunks = chunk_list(tracks, 50)

uri_to_duration_popularity = {}
for i, chunk in enumerate(chunks):
    print(i)
    try:
        tracks_info = sp.tracks(chunk)  # Consulta a la API
        for track in tracks_info.get('tracks', []):
            if track:
                uri = track.get('uri')
                duration = track.get('duration_ms')
                popularity = track.get('popularity')
                uri_to_duration_popularity[uri] = (duration, popularity)
            else:
                print("⚠️  Track no encontrado o no disponible.")
    except Exception as e:
        print(f"❌ Error al procesar el chunk {i + 1}: {e}")

# Creamos las nuevas columnas aplicando las funciones
submission['duration_ms'] = submission['spotify_track_uri'].apply(get_duration)
submission['popularity'] = submission['spotify_track_uri'].apply(get_popularity)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [62]:
data

,Unnamed: 0,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,shuffle,TARGET,hour,day_of_week,month,...,reason_clickrow,reason_fwdbtn,reason_playbtn,reason_remote,reason_trackdone,reason_trackerror,reason_unknown,reason_nan,duration_ms,popularity
0,110163,Crazy For U,Big Time Rush,24/seven,spotify:track:3jFfr89lnSmb4QBtfG8JBP,False,False,5,6,3,...,1,0,0,0,0,0,0,0,187120.0,44.0
1,66026,Nada Personal - Remasterizado 2007,Soda Stereo,Me Verás Volver (Hits & Más),spotify:track:09TTeexnlKewZdjOak2sV2,True,False,10,0,6,...,0,0,0,0,1,0,0,0,292573.0,55.0
2,116790,Good Times,CHIC,Risqué,spotify:track:0G3fbPbE1vGeABDEZF0jeG,True,True,2,6,7,...,0,0,0,0,1,0,0,0,495400.0,68.0
3,18431,Verano del 92,Los Piojos,3er Arco,spotify:track:1NXvuBAq48QrxRFQZVmORQ,False,True,4,6,9,...,0,0,0,0,1,0,0,0,430960.0,0.0
4,82941,Simpatico,Ekko Park,Simpatico,spotify:track:2gYJY0sIx1ErgTIha2nPRg,True,False,18,3,7,...,0,0,0,0,1,0,0,0,214848.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100139,78756,Losing My Religion,R.E.M.,In Time: The Best Of R.E.M. 1988-2003,spotify:track:12axV6NUqaYH3yFUWwArzr,True,False,20,3,4,...,0,0,0,0,1,0,0,0,269000.0,1.0
100140,12585,Cerca De La Revolucion,Charly García,Piano Bar,spotify:track:66grIvFLGrI4tNhggO2DAd,True,False,21,2,11,...,0,0,0,0,1,0,0,0,280546.0,55.0
100141,93960,A Los Jóvenes De Ayer - Remastered 2012,Serú Girán,Bicicleta,spotify:track:6YAl320SfxLO4rVIZTFKqG,True,False,1,4,1,...,0,0,0,0,1,0,0,0,569453.0,38.0
100142,74339,heat not hot,Serengeti,heat not hot,spotify:track:0rKLO1hXpmoIthQgJKoczN,True,False,17,2,5,...,0,0,0,0,1,0,0,0,242845.0,31.0


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [32]:
# Entrenamiento y evaluación del modelo Random Forest
N_TREES = 500

rf = RandomForestClassifier(n_estimators=N_TREES, n_jobs=-1, random_state=6789, verbose=1, oob_score=True)
rf.fit(data.drop(columns=['TARGET', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'spotify_track_uri']), data['TARGET'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished


RandomForestClassifier(n_estimators=500, n_jobs=-1, oob_score=True,
                       random_state=6789, verbose=1)

In [63]:
prediccion = rf.predict_proba(submission.drop(columns=['master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'spotify_track_uri']))[:, 1]

f_out = open('submission2.csv', 'w')
f_out.write('ID,TARGET\n')
for i in range(len(prediccion)):
    f_out.write(f'{submission.iloc[i]["Unnamed: 0"]},{prediccion[i]}\n')



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    3.6s finished
